# Checking enviroment states and action sets

In [ ]:
import sys
import os.path
sys.path.append("..")
from PPO_Agent_with_intrinsic_reward_V2 import * #PPO_Agent_v2  PPO_Agent_with_Guided_AC
from ENV_DETAILS import *

In [ ]:
if os.name == 'nt':
    main_hyper_dir = "D:\\Artificial_Intelligence\\Portfolio\\RL_updated\\BipedalWalker\\" # Windows
    conda_python_exec = 'C:\\Users\\yanie\\anaconda3\\envs\\ai_dev\\python.exe '# Windows
    logs_dir = main_hyper_dir+'Hyperparam_tuning\\' # Windows
else:
    main_hyper_dir = "/media/n/NewDisk/Artificial_Intelligence/Portfolio/RL_updated/BipedalWalker/" # Linux
    conda_python_exec = '/home/n/anaconda3/envs/ai_dev/bin/python '# Linux
    logs_dir = main_hyper_dir+'Hyperparam_tuning_ppo/' # Linux

ENV = "Ant-v2"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [ ]:

env = gym.make(ENV)
env


In [ ]:
s = env.observation_space.sample()
s

In [ ]:
env.observation_space, env.observation_space.shape

In [ ]:
s = env.reset()
s

In [ ]:
s = env.reset()
env.action_space, env.action_space

### Training/Hyperparam run

In [ ]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "Training/fit_PPO_intrinsic_reward/"

In [ ]:
if HYPERPARAM_TUNING:

    dir = r"Hyperparam_kt_ppo"
    project_name = "keras_tunning_ppo_intrinsic_reward"
    

    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/ppo_intrinsic_reward/", evaluation_epoch = env._max_episode_steps, training_steps = 1000000,
                                sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
                discount_min = 0.95, discount_max = 0.99,
                gae_min = 0.90, gae_max = 0.96, policy_clip =0.2,scaling_factor_reward = 0.1,
                lr_actor_min = 0.00001, lr_actor_max = 0.001,
                lr_critic_min = 0.00001, lr_critic_max = 0.001,
                entropy_factor_min = 0.01, entropy_factor_max = 0.5,
                dense_min = 32, dense_max = 256,
                environment_name=ENV, num_layers_act = 3, num_layers_crit =3, num_layers_model = 3, training_epoch = 50,
                memory_size = 10#env._max_episode_steps
                #memory_size_max= env._max_episode_steps
                ),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 40,
            # distribution_strategy= strategy,
            directory=dir,
            project_name=project_name
        )
    tuner.search(x=[0], y=[1])
else : 
    
    print("Acquiring parameters ....")
    writer= "logs_hyper/ppo_intrinsic_reward/V0_"#Training/fit_ppo/"

    training_steps = 1000000
    entropy_factor = 0.05
    discount = 0.99
    dense_units_actor = [128]#64, 32]
    num_layers_actor = 1
    dense_units_critic = [128]#64,32]
    num_layers_crit =1
    num_layer_m = 1
    dense_units_model = [32]

    model = run_training(training_steps,  discount, dense_units_actor,  dense_units_critic, dense_units_model,  num_layers_actor, num_layers_crit, num_layer_m, writer, 
                    environment_name = ENV, return_agent = True, lr_actor= 0.00001, lr_critic= 0.00001, 
                    gae_lambda = 0.95, entropy_coeff = entropy_factor, policy_clip = 0.2, training_epoch = 50)



In [ ]:
exploration_tech = "soft"
hyperparam_combination=[]
for trials in tuner.oracle.get_best_trials(num_trials=10):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

In [ ]:
val_env = gym.make(ENV)#, render_mode = "rgb_array"
dir = r"Hyperparam_kt_ppo"
for trials in tuner.oracle.get_best_trials(num_trials=50):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)    
    training_steps = 2000000
    entropy_factor = trials.hyperparameters.values["entropy_coeff"]
    discount = trials.hyperparameters.values["discount"]
    gae = trials.hyperparameters.values["gae_lambda"]
    policy_clip = trials.hyperparameters.values["policy_clip"]
    scaling_factor_reward= trials.hyperparameters.values["scaling_factor_reward"]
    
    
    lr_actor=  trials.hyperparameters.values["lr_actor"]
    lr_critic=  trials.hyperparameters.values["lr_critic"]
    lr_model=  trials.hyperparameters.values["lr_model"]
    
    try:
        n_dense_layers_actor = trials.hyperparameters.values["n_dense_layers_actor"]
    except : 
        n_dense_layers_actor = 1
        
    try:
        n_dense_layers_critic = trials.hyperparameters.values["n_dense_layers_critic"]
    except:
        n_dense_layers_critic = 1
    

    dense_layers_actor = []
    for i in range(n_dense_layers_actor):
        dense_layers_actor.append(trials.hyperparameters.values['dense_units_act_'+str(i)])

    dense_layers_critic = []
    for i in range(n_dense_layers_critic):
        dense_layers_critic.append(trials.hyperparameters.values['dense_units_crit_'+str(i)])

    
    n_dense_layers_model = 1
    dense_layers_model = []
    for i in range(n_dense_layers_model):
        dense_layers_model.append(trials.hyperparameters.values['n_dense_layers_model'+str(i)])

    model = run_training(
        training_steps = training_steps,   
            discount = discount,
            dense_units_act = dense_layers_actor, 
            dense_units_crit = dense_layers_critic,
            dense_units_model = dense_layers_model,
            num_layer_a = n_dense_layers_actor,
            num_layer_c = n_dense_layers_critic,
            num_layer_m = n_dense_layers_model,
            writer = writer,  
            save_factor=50000, 
            sucess_criteria_epochs =SUCESS_CRITERIA_EPOCH, 
            sucess_criteria_value = SUCESS_CRITERIA_VALUE,
            environment_name=ENV,
            reward_scaler = 1, 
            evaluation_epoch = env._max_episode_steps,
            return_agent = True,
            lr_actor= lr_actor, 
            lr_critic= lr_critic,
            lr_model= lr_model,
            gae_lambda=gae,
            training_epoch= 200,
            entropy_coeff= entropy_factor,
            policy_clip = policy_clip,
            memory_size= env._max_episode_steps,
            id = int(trials.trial_id),
            scaling_factor_reward = scaling_factor_reward)
        
final_rewards = final_evaluation(model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./ppo_"+exploration_tech+"_video.mp4", env= ENV)

In [ ]:
env_model = tuner.get_best_models()[0]
final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./ppo_"+exploration_tech+"_video.mp4", env= ENV)

In [ ]:
print("Final mean reward '",exploration_tech,"':", np.mean(final_rewards))


In [ ]:
final_rewards